In [28]:
import numpy as np
from sklearn import preprocessing

raw_data=np.loadtxt('Audiobooks-data.csv',delimiter=',')

unscaled_inputs_all=raw_data[:,1:-1]
targets_all=raw_data[:,-1]


In [50]:
unscaled_inputs_all.shape

(14084, 10)

In [59]:
num_one_targets=np.sum(targets_all)

zero_targets_counter=0

indices_to_remove=[]

for i in range(targets_all.shape[0]):
    if targets_all[i]==0:
        zero_targets_counter+=1
        if zero_targets_counter>num_one_targets:
            indices_to_remove.append(i)
            
unscaled_inputs_equal_priors=np.delete(unscaled_inputs_all,indices_to_remove,axis=0)
targets_equal_priors=np.delete(targets_all,indices_to_remove,axis=0)


In [30]:
scaled_inputs=preprocessing.scale(unscaled_inputs_equal_priors)

In [68]:
shuffled_indices =np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)
shuffled_inputs=scaled_inputs[shuffled_indices]
shuffled_targets=targets_equal_priors[shuffled_indices]

In [69]:
samples_count=shuffled_inputs.shape[0]

train_samples_count=int(0.8*samples_count)
validation_samples_count=int(0.1*samples_count)
test_samples_count=int(samples_count-train_samples_count-validation_samples_count)

train_inputs=shuffled_inputs[:train_samples_count]
train_targets=shuffled_targets[:train_samples_count]

validation_inputs=shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets=shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs=shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets=shuffled_targets[train_samples_count+validation_samples_count:]

print(np.sum(train_targets),train_samples_count,np.sum(train_targets)/train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1780.0 3579 0.4973456272701872
242.0 447 0.5413870246085011
215.0 448 0.4799107142857143


In [48]:
np.savez('Audiobooks_data_train', inputs=train_inputs,outputs=train_targets)
np.savez('Audiobooks_data_validation',inputs=validation_inputs,outputs=validation_targets)
np.savez('Audiobooks_data_test',inputs=test_inputs,outputs=test_targets)

In [49]:
import numpy as np
import tensorflow as tf

In [52]:
input_size=10
output_size=2
hidden_layer_size=30

model=tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size,activation='softmax')
    
    
])


In [53]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [55]:
MAX_SPOCHS=100
batch_size=100

early_stoppings=tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs,train_targets,validation_data=(validation_inputs,validation_targets),callbacks=[early_stoppings],epochs=MAX_SPOCHS,batch_size=batch_size,verbose=1)

Epoch 1/100
36/36 [==============================] - 0s 3ms/step - loss: 0.3795 - accuracy: 0.7949 - val_loss: 0.3589 - val_accuracy: 0.8098
Epoch 2/100
36/36 [==============================] - 0s 1ms/step - loss: 0.3702 - accuracy: 0.7994 - val_loss: 0.3560 - val_accuracy: 0.7875
Epoch 3/100
36/36 [==============================] - 0s 2ms/step - loss: 0.3613 - accuracy: 0.8047 - val_loss: 0.3469 - val_accuracy: 0.8076
Epoch 4/100
36/36 [==============================] - 0s 2ms/step - loss: 0.3595 - accuracy: 0.8083 - val_loss: 0.3600 - val_accuracy: 0.7763
Epoch 5/100
36/36 [==============================] - 0s 2ms/step - loss: 0.3513 - accuracy: 0.8139 - val_loss: 0.3333 - val_accuracy: 0.8121
Epoch 6/100
36/36 [==============================] - 0s 2ms/step - loss: 0.3560 - accuracy: 0.8041 - val_loss: 0.3407 - val_accuracy: 0.8188
Epoch 7/100
36/36 [==============================] - 0s 2ms/step - loss: 0.3465 - accuracy: 0.8120 - val_loss: 0.3368 - val_accuracy: 0.7875


In [56]:
test_loss,test_accuracy=model.evaluate(test_inputs,test_targets)

14/14 [==============================] - 0s 1ms/step - loss: 0.3659 - accuracy: 0.7857
